# List schools that haven't been successfully webscraped 

Author: Jaren Haber <br>
Project Manager: Jaren Haber, PhD Candidate <br>
Contact: jhaber@berkeley.edu

Institution: University of California, Berkeley <br>
Program: Undergraduate Research Apprentice Program (URAP) <br>

Date created: November 8th, 2018 <br>
Last modified: November 8th, 2018

Description: For the purposes of improving our URL list and therefore our data collection, this script identifies two groups: schools with URLs that were not webscraped, and schools open in 2015 for which we didn't collect URLs (and therefore were also not webscraped). It produces a data file for each of these two groups of schools. Next steps: For the first group, the URLs will need to be hand-cleaned; for the second, URLs will need to be collected by hand.

## Initialize

In [1]:
# Import packages
import pandas as pd
import numpy as np
import gc # For managing garbage collector

In [2]:
# Load data
gc.disable()
charterdf = pd.read_pickle('../../nowdata/charters_2015.pkl')
gc.enable()

print(charterdf.shape)

(10965, 704)


In [3]:
charterdf["NCESSCH"] = charterdf["NCESSCH"].astype(float) # Convert to float type (just in case)

In [4]:
# Sanity check: Show number of duplicates by NCESSCH:
print("# duplicates by NCESSCH: ", sum(charterdf.duplicated(subset=["NCESSCH"])))

# duplicates by NCESSCH:  0


## Identify schools open in 2015-16 but NO URL

No URL: `URL`.isna() or `URL`==""

School open in 2015-16: <br>
- `open:` SY_STATUS15 == 1,3,4,5 or 8  <br>
- `closed:` SY_STATUS15 == 2,6 or 7

In [5]:
# Keep only those columns useful for URL collection:
nourl_df = charterdf[["SEARCH16", "URL", "NCESSCH", "SCHNAM16", "ADDRESS16", "SY_STATUS16", "SY_STATUS15", "LEVEL", "STABR", "WEBSITE", "YEAR_CLOSED", "SCH_NAME", "YEAR_OPENED"]]
print(nourl_df.shape)
list(nourl_df)

(10965, 13)


['SEARCH16',
 'URL',
 'NCESSCH',
 'SCHNAM16',
 'ADDRESS16',
 'SY_STATUS16',
 'SY_STATUS15',
 'LEVEL',
 'STABR',
 'WEBSITE',
 'YEAR_CLOSED',
 'SCH_NAME',
 'YEAR_OPENED']

In [6]:
# Keep ONLY those schools missing URLs (keep those with URL==0, which means school has no website, whether closed or whatever):
nourl1 = nourl_df[pd.isnull(nourl_df['URL'])] # Store rows where URL missing
nourl2 = nourl_df[nourl_df["URL"] == ''] # Store if URL an empty string

# Now concatenate these stored rows into one DF:
nourl_df = pd.concat([nourl1, nourl2])
print(nourl_df.shape)

(3828, 13)


In [7]:
# Filter to only open schools
filtered_STATUS = (nourl_df['SY_STATUS15'] == 1) | (nourl_df['SY_STATUS15'] == 3) | (nourl_df['SY_STATUS15'] == 4) | (nourl_df['SY_STATUS15'] == 5) | (nourl_df['SY_STATUS15'] == 8)

nourl_df = nourl_df.loc[filtered_STATUS]
print(nourl_df.shape)
nourl_df.head()

(402, 13)


,SEARCH16,URL,NCESSCH,SCHNAM16,ADDRESS16,SY_STATUS16,SY_STATUS15,LEVEL,STABR,WEBSITE,YEAR_CLOSED,SCH_NAME,YEAR_OPENED
146,NORTHERN AZ ACADEMY FOR CAREER DEV. - WINSLOW ...,NaN,4.000770e+10,NORTHERN AZ ACADEMY FOR CAREER DEV. - WINSLOW,"502 Airport Road , Winslow AZ 86047.0",2.0,1.0,3,AZ,http://www.naacharter.org/,2016.0,NaN,1998.0
260,PEPE BARRON ACADEMY 2797 North Introspect Dri...,NaN,4.001250e+10,PEPE BARRON ACADEMY,"2797 North Introspect Drive , Tucson AZ 85745.0",1.0,1.0,3,AZ,NaN,NaN,NaN,1998.0
292,"DISCOVERY PLUS ACADEMY 852 West 250 North , P...",NaN,4.001410e+10,DISCOVERY PLUS ACADEMY,"852 West 250 North , Pima AZ 85543.0",1.0,1.0,1,AZ,http://myweb.cableone.net/discplus1/,NaN,NaN,1998.0
386,"THE SHELBY SCHOOL 249 W. Standage Drive , Pay...",NaN,4.002140e+10,THE SHELBY SCHOOL,"249 W. Standage Drive , Payson AZ 85541.0",1.0,1.0,4,AZ,NaN,NaN,NaN,1999.0
516,PEPE BARRON MIDDLE SCHOOL 2797 N. Introspect ...,NaN,4.002960e+10,PEPE BARRON MIDDLE SCHOOL,"2797 N. Introspect Drive , Tucson AZ 85745.0",1.0,1.0,1,AZ,NaN,NaN,NaN,2000.0


In [8]:
print("# duplicates by NCESSCH: ", sum(nourl_df.duplicated(subset='NCESSCH', keep='first')))

# Eliminate duplicates in pubschool data:
print("Removing duplicates...")
nourl_df = nourl_df.drop_duplicates(subset="NCESSCH", keep='first')

print("NEW # duplicates by NCESSCH: ", sum(nourl_df.duplicated(subset='NCESSCH', keep='first')))

# duplicates by NCESSCH:  0
Removing duplicates...
NEW # duplicates by NCESSCH:  0


In [9]:
print("#rows, #cols WITH empty by NCESSCH: ", nourl_df.shape)
nourl_df.dropna(subset=["NCESSCH"], inplace=True)
print("#rows, #cols WITHOUT empty by NCESSCH: ", nourl_df.shape)

#rows, #cols WITH empty by NCESSCH:  (402, 13)
#rows, #cols WITHOUT empty by NCESSCH:  (402, 13)


In [10]:
list(nourl_df)

['SEARCH16',
 'URL',
 'NCESSCH',
 'SCHNAM16',
 'ADDRESS16',
 'SY_STATUS16',
 'SY_STATUS15',
 'LEVEL',
 'STABR',
 'WEBSITE',
 'YEAR_CLOSED',
 'SCH_NAME',
 'YEAR_OPENED']

In [11]:
nourl_df

,SEARCH16,URL,NCESSCH,SCHNAM16,ADDRESS16,SY_STATUS16,SY_STATUS15,LEVEL,STABR,WEBSITE,YEAR_CLOSED,SCH_NAME,YEAR_OPENED
146,NORTHERN AZ ACADEMY FOR CAREER DEV. - WINSLOW ...,NaN,4.000770e+10,NORTHERN AZ ACADEMY FOR CAREER DEV. - WINSLOW,"502 Airport Road , Winslow AZ 86047.0",2.0,1.0,3,AZ,http://www.naacharter.org/,2016.0,NaN,1998.0
260,PEPE BARRON ACADEMY 2797 North Introspect Dri...,NaN,4.001250e+10,PEPE BARRON ACADEMY,"2797 North Introspect Drive , Tucson AZ 85745.0",1.0,1.0,3,AZ,NaN,NaN,NaN,1998.0
292,"DISCOVERY PLUS ACADEMY 852 West 250 North , P...",NaN,4.001410e+10,DISCOVERY PLUS ACADEMY,"852 West 250 North , Pima AZ 85543.0",1.0,1.0,1,AZ,http://myweb.cableone.net/discplus1/,NaN,NaN,1998.0
386,"THE SHELBY SCHOOL 249 W. Standage Drive , Pay...",NaN,4.002140e+10,THE SHELBY SCHOOL,"249 W. Standage Drive , Payson AZ 85541.0",1.0,1.0,4,AZ,NaN,NaN,NaN,1999.0
516,PEPE BARRON MIDDLE SCHOOL 2797 N. Introspect ...,NaN,4.002960e+10,PEPE BARRON MIDDLE SCHOOL,"2797 N. Introspect Drive , Tucson AZ 85745.0",1.0,1.0,1,AZ,NaN,NaN,NaN,2000.0
519,VISIONS UNLIMITED ACADEMY 1275 East Barney La...,NaN,4.002990e+10,VISIONS UNLIMITED ACADEMY,"1275 East Barney Lane , Benson AZ 85602.0",1.0,1.0,1,AZ,VisionsUnlimitedAcademy.com,NaN,NaN,2000.0
545,SOUTHSIDE COMMUNITY SCHOOL 2701 S. Campbell A...,NaN,4.003180e+10,SOUTHSIDE COMMUNITY SCHOOL,"2701 S. Campbell Ave , Tucson AZ 85713.0",1.0,1.0,4,AZ,NaN,NaN,NaN,2000.0
632,CARPE DIEM COLLEGIATE HIGH SCHOOL DBA CARPE DI...,NaN,4.003810e+10,CARPE DIEM COLLEGIATE HIGH SCHOOL DBA CARPE DI...,"3777 W. 22nd Lane , Yuma AZ 85364.0",1.0,1.0,4,AZ,NaN,NaN,NaN,2002.0
633,"ISCHOOL2020 3777 W 22nd Lane , Yuma AZ 85364.0",NaN,4.003810e+10,ISCHOOL2020,"3777 W 22nd Lane , Yuma AZ 85364.0",1.0,1.0,3,AZ,NaN,NaN,NaN,2007.0
647,LIFELONG LEARNING ACADEMY 3295 W. Orange Grov...,NaN,4.003930e+10,LIFELONG LEARNING ACADEMY,"3295 W. Orange Grove , Tucson AZ 85741.0",1.0,1.0,1,AZ,NaN,NaN,NaN,2002.0


## Identify schools with URLs that weren't webscraped

Not scraped: `WEBTEXT`  length == 0  

Has URL: `URL`.isnull()==False (or use dropna) and `URL`!=""

In [12]:
# Load data
gc.disable()
charterdf = pd.read_pickle('../../nowdata/charters_2015.pkl')
gc.enable()

print(charterdf.shape)

(10965, 704)


In [72]:
# Drop schools missing URLs (keep those with URL==0, which means school has no website, whether closed or whatever):
charterdf = charterdf.dropna(subset = ['URL']) # drop if URL missing
charterdf = charterdf[charterdf["URL"] != ''] # drop if URL an empty string
print(charterdf.shape)
list(charterdf)

(7137, 702)


['NCESSCH',
 'URL',
 'LAT1516',
 'LON1516',
 'AM',
 'AS',
 'BL',
 'HI',
 'HP',
 'TR',
 'TOTFRL',
 'CHARTER_TEXT',
 'WEBSITE',
 'MEMBER',
 'LEVEL',
 'LOCALE15',
 'LEAID',
 'LSTREET1',
 'LSTREET2',
 'LSTREET3',
 'LCITY',
 'LSTATE',
 'LZIP',
 'CMO_NAME',
 'CMO_MEMSUM',
 'SCH_NAME',
 'CMO_STATE',
 'CMO_SCHNUM',
 'CMO_URL',
 'CMO_NUMSTATES',
 'CMO_ALLSTATES',
 'CMO_SECTOR',
 'CMO_NUMSTUDENTS_CREDO17',
 'CMO_TYPE',
 'SURVYEAR',
 'FIPST',
 'STABR',
 'SEANAME',
 'ST_LEAID',
 'SCHID',
 'ST_SCHID',
 'MSTREET1',
 'MSTREET2',
 'MSTREET3',
 'MCITY',
 'MSTATE',
 'MZIP',
 'MZIP4',
 'PHONE',
 'LZIP4',
 'UNION',
 'OUT_OF_STATE_FLAG',
 'SCH_TYPE_TEXT',
 'SCH_TYPE',
 'RECON_STATUS',
 'GSLO',
 'GSHI',
 'VIRTUAL',
 'BIES',
 'SY_STATUS_TEXT',
 'SY_STATUS',
 'UPDATED_STATUS_TEXT',
 'UPDATED_STATUS',
 'EFFECTIVE_DATE',
 'G13OFFERED',
 'AEOFFERED',
 'UGOFFERED',
 'NOGRADES',
 'CHARTAUTH1',
 'CHARTAUTHN1',
 'CHARTAUTH2',
 'CHARTAUTHN2',
 'IGOFFERED',
 'FRELCH',
 'REDLCH',
 'AE',
 'TOTAL',
 'AMALM',
 'AMALF',
 '

In [73]:
# Keep only rows that are missing WEBTEXT (scrape failed):
notext1 = charterdf['WEBTEXT'].isnull() # Store rows where WEBTEXT missing
notext2 = charterdf[charterdf["WEBTEXT"].apply(len) == 0] # Store rows where NO WEBTEXT (0 pages) was gathered

# To prepare for parsing deeper into WEBTEXT to look for null results, now exclude schools with no WEBTEXT
charterdf = charterdf[charterdf["WEBTEXT"].apply(len) > 0] # Keep only rows with some WEBTEXT (at least one page)
charterdf = charterdf.dropna(subset = ['WEBTEXT']) # drop if WEBTEXT missing

notext3 = charterdf[charterdf["WEBTEXT"].apply(lambda school: len(school[0][3])) == 0] # Store rows where first scraped page is empty
notext4 = charterdf[charterdf["WEBTEXT"].apply(lambda school: sum([len(page[3]) for page in school])) == 0] # Store rows where all pages are empty

In [74]:
print("# rows, #cols: ", notext1.shape)
print("# rows, #cols: ", notext2.shape)
print("# rows, #cols: ", notext3.shape)
print("# rows, #cols: ", notext4.shape)

# rows, #cols:  (7137,)
# rows, #cols:  (275, 702)
# rows, #cols:  (442, 702)
# rows, #cols:  (399, 702)


In [75]:
# Now concatenate these stored rows into one DF:
hasurl_df = pd.concat([notext1, notext2, notext3, notext4])
print(hasurl_df.shape)

(8253, 703)


/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [76]:
# Keep only those columns useful for URL collection:
hasurl_df = hasurl_df[["SEARCH16", "URL", "NCESSCH", "SCHNAM16", "ADDRESS16", "SY_STATUS16", "SY_STATUS15", "LEVEL", "STABR", "WEBSITE", "YEAR_CLOSED", "SCH_NAME", "YEAR_OPENED"]]

In [77]:
print("# duplicates by NCESSCH: ", sum(hasurl_df.duplicated(subset='NCESSCH', keep='first')))

# Eliminate duplicates in pubschool data:
print("Removing duplicates...")
hasurl_df = hasurl_df.drop_duplicates(subset="NCESSCH", keep='first')

print("NEW # duplicates by NCESSCH: ", sum(hasurl_df.duplicated(subset='NCESSCH', keep='first')))

# duplicates by NCESSCH:  7535
Removing duplicates...
NEW # duplicates by NCESSCH:  0


In [87]:
print("#rows, #cols WITH empty by NCESSCH: ", hasurl_df.shape)
hasurl_df.dropna(subset=["NCESSCH"], inplace=True)
print("#rows, #cols WITHOUT empty by NCESSCH: ", hasurl_df.shape)

#rows, #cols WITH empty by NCESSCH:  (717, 13)
#rows, #cols WITHOUT empty by NCESSCH:  (717, 13)


In [92]:
list(hasurl_df)

['SEARCH16',
 'URL',
 'NCESSCH',
 'SCHNAM16',
 'ADDRESS16',
 'SY_STATUS16',
 'SY_STATUS15',
 'LEVEL',
 'STABR',
 'WEBSITE',
 'YEAR_CLOSED',
 'SCH_NAME',
 'YEAR_OPENED']

In [84]:
hasurl_df

,SEARCH16,URL,NCESSCH,SCHNAM16,ADDRESS16,SY_STATUS16,SY_STATUS15,LEVEL,STABR,WEBSITE,YEAR_CLOSED,SCH_NAME,YEAR_OPENED
1,"AYAPRUN ELITNAURVIK 1010 Fourth Avenue , Beth...",https://education.alaska.gov/DOE_Rolodex/Schoo...,2.000010e+10,AYAPRUN ELITNAURVIK,"1010 Fourth Avenue , Bethel AK 99559.0",1.0,1.0,1,AK,http://www.lksd.org/lksd/,NaN,NaN,1999.0
5,FAMILY PARTNERSHIP CHARTER SCHOOL 401 E Firew...,https://education.alaska.gov/DOE_Rolodex/Schoo...,2.001800e+10,FAMILY PARTNERSHIP CHARTER SCHOOL,"401 E Fireweed Lane Suite 100 , Anchorage AK ...",1.0,1.0,4,AK,http://www.asdk12.org/aboutschools/fpcs/school...,NaN,NaN,1998.0
12,RILKE SCHULE CHARTER SCHOOL 650 W Internation...,https://education.alaska.gov/DOE_Rolodex/Schoo...,2.001800e+10,RILKE SCHULE CHARTER SCHOOL,"650 W International Airport Rd , Anchorage AK...",1.0,1.0,1,AK,http://www.asdk12.org/aboutschools/rilkeschule...,NaN,NaN,2007.0
16,AURORA BOREALIS CHARTER SCHOOL 705 Frontage R...,https://education.alaska.gov/DOE_Rolodex/Schoo...,2.003900e+10,AURORA BOREALIS CHARTER SCHOOL,"705 Frontage Rd Suite A , Kenai AK 99611.0",1.0,1.0,1,AK,http://www.kpbsd.k12.ak.us/,NaN,NaN,1998.0
28,ANVIL CITY SCIENCE ACADEMY Mile 3.5 Nome-Tell...,http://nome.nosd.schoolaccess.net/~acsa/,2.005700e+10,ANVIL CITY SCIENCE ACADEMY,"Mile 3.5 Nome-Teller Highway , Nome AK 99762.0",1.0,1.0,2,AK,http://nome.nosd.schoolaccess.net/~acsa/,NaN,NaN,1998.0
32,EFFIE KOKRINE CHARTER SCHOOL 601 Loftus Road ...,http://ekc.k12northstar.org/,2.006000e+10,EFFIE KOKRINE CHARTER SCHOOL,"601 Loftus Road , Fairbanks AK 99709.0",1.0,1.0,3,AK,http://www.k12northstar.org,NaN,NaN,2005.0
143,KINGMAN ACADEMY OF LEARNING - HIGH SCHOOL 342...,http://kaolaz.org/calendar.html,4.000740e+10,KINGMAN ACADEMY OF LEARNING - HIGH SCHOOL,"3420 N. Burbank Ave. , Kingman AZ 86409.0",1.0,1.0,3,AZ,http://www.kaolaz.org,NaN,KINGMAN ACADEMY OF LEARNING - HIGH SCHOOL,2000.0
381,CANYON VIEW PREP ACADEMY 9030 E Florentine Ro...,http://www.canyonviewprep.org/,4.002090e+10,CANYON VIEW PREP ACADEMY,"9030 E Florentine Road , Prescott Valley AZ 8...",1.0,1.0,3,AZ,canyonviewprep.org,NaN,NaN,2013.0
443,PARADISE EDUCATION CENTER 15533 W. Paradise L...,http://www.pecschools.org/,4.002410e+10,PARADISE EDUCATION CENTER,"15533 W. Paradise Ln. , Surprise AZ 85374.0",1.0,1.0,1,AZ,paradiseschools.org,NaN,NaN,1999.0
444,PARADISE HONORS HIGH SCHOOL 12775 N. 175th Av...,http://pecschools.org/,4.002410e+10,PARADISE HONORS HIGH SCHOOL,"12775 N. 175th Avenue , Surprise AZ 85374.0",1.0,1.0,3,AZ,http://www.paradiseschools.org,NaN,NaN,2008.0


## Save output

In [86]:
# Save DataFrame as CSV to data-management, web-scraping, and scrapy-cluster repos:
hasurl_df.to_csv('../../data_management/data/charters_unscraped_hasURL_2015.csv', index=False)
nourl_df.to_csv('../../data_management/data/charters_unscraped_noURL_2015.csv', index=False)

hasurl_df.to_csv('../../web_scraping/data/charters_unscraped_hasURL_2015.csv', index=False)
nourl_df.to_csv('../../web_scraping/data/charters_unscraped_noURL_2015.csv', index=False)

hasurl_df.to_csv('../../scrapy-cluster/kafka-monitor/charter_urls_2016_unscraped.csv', index=False)
nourl_df.to_csv('../../scrapy-cluster/kafka-monitor/charter_urls_2016_unscraped.csv', index=False)